Устанавливаем allennlp, чтобы выгрузить оттуда эмбеддинги ELMo:

In [2]:
!pip install flair

  Using cached https://files.pythonhosted.org/packages/3a/83/e74092e7f24a08d751aa59b37a9fc572b2e4af3918cb66f7766c3affb1b4/transformers-3.5.1-py3-none-any.whl
Processing /root/.cache/pip/wheels/6f/1b/ec/6c71a1eb823df7f850d956b2d8c50a6d49c191e1063d73b9be/overrides-3.0.0-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4c/34/b39eb9994bc3c999270b69c9eea40ecc6f0e97991dba28282b9fd32d44ee/tokenizers-0.9.3-cp36-cp36m-manylinux1_x86_64.whl
ERROR: allennlp 2.0.1 has requirement overrides==3.1.0, but you'll have overrides 3.0.0 which is incompatible.
ERROR: allennlp 2.0.1 has requirement transformers<4.3,>=4.1, but you'll have transformers 3.5.1 which is incompatible.
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Found existing installation: transformers 4.2.2
    Uninstalling transformers-4.2.2:
      Successfully uninstalled transformers-4.2.2
  Found existing installation: overrides 3

In [1]:
# Uncomment if you need to download data
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/test.conll

--2021-02-09 17:29:59--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125059 (122K) [text/plain]
Saving to: ‘train.conll.3’

train.conll.3       100%[===================>] 122.13K  --.-KB/s    in 0.03s   

2021-02-09 17:29:59 (4.66 MB/s) - ‘train.conll.3’ saved [125059/125059]

--2021-02-09 17:29:59--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14109 (14K) [

In [4]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings, BytePairEmbeddings, TransformerWordEmbeddings

In [5]:
import flair
import torch
# uncomment for cpu-only training
#flair.device = torch.device('cpu')

In [6]:
# 1. get the corpus
corpus: Corpus = ColumnCorpus(
    "./", column_format={0:'text', 1:'ner'},
    train_file='train.conll', dev_file='dev.conll', test_file='test.conll',
)
print(corpus)

2021-02-09 17:01:02,680 Reading data from .
2021-02-09 17:01:02,683 Train: train.conll
2021-02-09 17:01:02,685 Dev: dev.conll
2021-02-09 17:01:02,687 Test: test.conll
Corpus: 1966 train + 219 dev + 547 test sentences


In [7]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
#embedding_types = [
 #   BytePairEmbeddings('multi')
#]

#embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

Dictionary with 14 tags: <unk>, O, B-SINGER, I-SINGER, B-SONG, B-FILM, I-SONG, I-FILM, B-COMPOSER, I-COMPOSER, B-BOOK, I-BOOK, <START>, <STOP>


Автоматически подбираем гиперпарметры:

In [8]:
from hyperopt import hp
from flair.hyperparameter.param_selection import SearchSpace, Parameter

In [22]:
search_space = SearchSpace()
search_space.add(Parameter.EMBEDDINGS, hp.choice, options=[
    StackedEmbeddings(embeddings=[BytePairEmbeddings('multi'), FlairEmbeddings('news-forward'), TransformerWordEmbeddings('bert-base-multilingual-cased'),TransformerWordEmbeddings('roberta-base')])
])
search_space.add(Parameter.HIDDEN_SIZE, hp.choice, options=[64, 128, 256])
search_space.add(Parameter.LEARNING_RATE, hp.choice, options=[0.05, 0.1, 0.15, 0.2])
search_space.add(Parameter.MINI_BATCH_SIZE, hp.choice, options=[8, 16, 32])

Setting dim=300 for multilingual BPEmb


Обучение:

In [ ]:
from flair.hyperparameter.param_selection import SequenceTaggerParamSelector, OptimizationValue


optimizer = SequenceTaggerParamSelector(
        corpus, "ner", './results', max_epochs=2
    )
optimizer.optimize(search_space, max_evals=3)

  0%|          | 0/50 [00:00<?, ?it/s, best loss: ?]2021-02-09 17:23:31,791 ----------------------------------------------------------------------------------------------------
2021-02-09 17:23:31,792 Evaluation run: 1
2021-02-09 17:23:31,795 Evaluating parameter combination:
2021-02-09 17:23:31,801 	embeddings: StackedEmbeddings [0-bpe-multi-100000-50,1-/root/.flair/embeddings/news-forward-0.4.1.pt,TransformerWordEmbeddings(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=T

In [12]:
# 5. initialize sequence tagger
# from flair.models import SequenceTagger

# tagger: SequenceTagger = SequenceTagger(hidden_size=256,
#                                         embeddings=embeddings,
#                                         tag_dictionary=tag_dictionary,
#                                         tag_type=tag_type,
#                                         use_crf=True)

# # 6. initialize trainer
# from flair.trainers import ModelTrainer

# trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# # 7. start training
# history = trainer.train(
#     './models/baseline-charembeddings',
#     learning_rate=0.1,
#     mini_batch_size=32,
#     max_epochs=20
# )

2021-02-09 16:20:59,587 ----------------------------------------------------------------------------------------------------
2021-02-09 16:20:59,588 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): BytePairEmbeddings(model=0-bpe-multi-100000-50)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=600, out_features=600, bias=True)
  (rnn): LSTM(600, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=14, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-02-09 16:20:59,589 ----------------------------------------------------------------------------------------------------
2021-02-09 16:20:59,590 Corpus: "Corpus: 1966 train + 219 dev + 547 test sentences"
2021-02-09 16:20:59,591 ----------------------------------------------------------------------------------------------------
2021-02-09 16:20:59,592 Parameters:
2021-02-09 1